In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pyodbc
import pandas as pd
import os
import configparser as cp

In [2]:
config= cp.ConfigParser()
config.read('db_config.ini')

pwd = config['secrets']['PGSQL_Comp_Fin_PASS']
uid = config['secrets']['PGSQL_Comp_Fin_UID']

In [3]:
server = "localhost"
db = "test_Company_Financials"
port = "5432"
file_dir= "C:\\Users\\USER\\Documents\\Python Scripts\\ETL Company Financials"
tbl_name = "Company_Financials"

In [4]:
#Extract Function
def extract_data_csv(csv_file):
    try:
        df= pd.read_csv(csv_file)
        return df
    
    except:
        print('Cant find file')
         

In [5]:
# Transform Function

integer_columns= 'Units Sold'

float_columns = ['Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit']

date_columns = 'Date'

non_essential_columns = ['Month Number', 'Month Name', 'Year'] 



def transform_data_csv(extracted_data):
    
    
    try:
        df = extracted_data.copy()
        
        #select all the columns with string and object data types
        df_obj = df.select_dtypes(['object','string'])
        
        #using the strip method to remove all trailing zeroes in the object columns
        df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
        
        
        # Remove the leading and trailing spaces in the columns names
        df.columns = df.columns.astype(str)
        
        df.columns = df.columns.str.strip()
        
        
        #Remove and/or Replace the following special characters in the tables '$', '-', ','
        df = df.replace ({'\$-':'0', ',':'','\$':'', '\(':'-', '\)':''}, regex=True)
        
        
        #Change data types
        df[date_columns] = pd.to_datetime(df[date_columns])
        df[float_columns]= df[float_columns].astype(float)
        df[integer_columns] = df[integer_columns].astype(float)
        df[integer_columns] = df[integer_columns]. astype(int)
        
        
        # drop columns
        df = df.drop(non_essential_columns, axis=1)
                
        return  df
        
    except:
        print('failed to transform')

In [6]:
# s= extract_data_csv('Financials.csv')

# df = transform_data_csv(s)

# rows_imported=0
# engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')
# print(f'Important rows {rows_imported} to {rows_imported + len(df)}...')
# df.to_sql(f"{tbl_name}", engine, if_exists = 'replace', index=False)
        
      
    


In [7]:
# Load Function

def load_data_csv(transformed_data):
    try:
        
        df = transformed_data
              
        rows_imported=0
        
        engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')
        print(f'Important rows {rows_imported} to {rows_imported + len(df)}...')
        df.to_sql(f"{tbl_name}", engine, if_exists = 'replace', index=False)
        
        rows_imported += len(df)
        
       
    except:
        print('Failed to load Data')

In [9]:
s= extract_data_csv('Financials.csv')

h = transform_data_csv(s)
h
# t = load_data_csv(h)
# t

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date
0,Government,Canada,Carretera,None,1618,3.0,20.0,32370.0,0.00,32370.00,16185.0,16185.00,2014-01-01
1,Government,Germany,Carretera,None,1321,3.0,20.0,26420.0,0.00,26420.00,13210.0,13210.00,2014-01-01
2,Midmarket,France,Carretera,None,2178,3.0,15.0,32670.0,0.00,32670.00,21780.0,10890.00,2014-01-06
3,Midmarket,Germany,Carretera,None,888,3.0,15.0,13320.0,0.00,13320.00,8880.0,4440.00,2014-01-06
4,Midmarket,Mexico,Carretera,None,2470,3.0,15.0,37050.0,0.00,37050.00,24700.0,12350.00,2014-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Small Business,France,Amarilla,High,2475,260.0,300.0,742500.0,111375.00,631125.00,618750.0,12375.00,2014-01-03
696,Small Business,Mexico,Amarilla,High,546,260.0,300.0,163800.0,24570.00,139230.00,136500.0,2730.00,2014-01-10
697,Government,Mexico,Montana,High,1368,5.0,7.0,9576.0,1436.40,8139.60,6840.0,1299.60,2014-01-02
698,Government,Canada,Paseo,High,723,10.0,7.0,5061.0,759.15,4301.85,3615.0,686.85,2014-01-04
